# Tensorflow

### 0. pip & import

##### pip

In [47]:
pip install googletrans==4.0.0-rc1  # 번역
pip install transformers # Pytorch StableDiffusion
pip install diffusers # Pytorch StableDiffusion
pip install matplotlib # 이미지 나타내기

##### import

- 공통

In [3]:
import nltk
from googletrans import Translator

nltk.download('punkt')  # 토큰화
nltk.download('stopwords')  # 불용어 리스트

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\k9942\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k9942\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- Pytorch StableDiffusion

In [2]:
import torch, logging

## Imaging  library
from PIL import Image
from torchvision import transforms as tfms

## Basic libraries
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import shutil
import os

## Import the CLIP artifacts
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler

c:\mockup\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. googletrans

In [4]:
translator=Translator()

In [10]:
new_title =  "강가에서"

In [11]:
new_title_trans=translator.translate(new_title,dest='en').text
new_title_trans # 영어로 번역

'On the river'

In [9]:
prompt_str = 'photograph, ' + new_title_trans + ', landscape, 8k uhd'

- 토큰화

In [51]:
stopwords=nltk.corpus.stopwords.words('english') # 영어의 불용어 단어 리스트 (불용어는 모두 소문자)

tokens=nltk.word_tokenize(new_title_trans)  # 각 문장을 토큰화

tokens=[t for t in tokens if t.lower() not in stopwords] # 토큰화한 각 문장에서 불용어 제거
print(tokens)

['dark', 'alleyway']


In [52]:
remove_prompts = ['good', 'song', 'listen', 'hear'] # '듣기 좋은 노래' 번역, 토큰화 결과 제거

prompts = [item for item in tokens if item not in remove_prompts] # 기존의 토큰 리스트에서 삭제할 프롬프트 제거

additional_prompts = ['photograph', 'landscape', 'simple'] # 스타일 프롬프트 추가
prompts = prompts + additional_prompts
print(prompts)

['dark', 'alleyway', 'photograph', 'landscape', 'simple']


In [53]:
input_prompts=','.join(prompts)
print(input_prompts)  # input 형식에 맞춰 수정

# 파이토치- []로 감싸야 함
# 케라스, 텐서플로우- 그대로 넣어도 됨

dark,alleyway,photograph,landscape,simple


### 2. 이미지 생성

In [ ]:
## 토크나이저 및 인코더 초기화
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cuda")

## VAE 초기화
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", torch_dtype=torch.float16).to("cuda")

## 스케줄러 초기화 및 샘플링 스텝 수 설정
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)
scheduler.set_timesteps(50)

## U-Net 모델 초기화
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", torch_dtype=torch.float16).to("cuda")

## 도우미 함수
def load_image(p):
    '''
    지정된 경로에서 이미지를 불러오는 함수
    '''
    return Image.open(p).convert('RGB').resize((512,512))

def pil_to_latents(image):
    '''
    이미지를 latent(잠재)으로 변환하는 함수
    '''
    init_image = tfms.ToTensor()(image).unsqueeze(0) * 2.0 - 1.0
    init_image = init_image.to(device="cuda", dtype=torch.float16)
    init_latent_dist = vae.encode(init_image).latent_dist.sample() * 0.18215
    return init_latent_dist

def latents_to_pil(latents):
    '''
    latent(잠재)을 이미지로 변환하는 함수
    '''
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images

def text_enc(prompts, maxlen=None):
    '''
    텍스트 프롬프트를 임베딩으로 변환하는 함수
    '''
    if maxlen is None: maxlen = tokenizer.model_max_length
    inp = tokenizer(prompts, padding="max_length", max_length=maxlen, truncation=True, return_tensors="pt")
    return text_encoder(inp.input_ids.to("cuda"))[0].half()

def prompt_2_img(prompts, neg_prompts=None, g=7.5, seed=100, steps=70, dim=512, save_int=False):
    """
    이미지로 프롬프트를 변환하는 확산 프로세스
    """

    # 배치 크기 정의
    bs = len(prompts)

    # 텍스트 프롬프트를 임베딩으로 변환
    text = text_enc(prompts)

    # 부정적인 프롬프트 조건 추가
    if not neg_prompts: uncond =  text_enc([""] * bs, text.shape[1])
    # 무조건적인 프롬프트 추가, 생성 프로세스를 돕는 데 도움이 됨
    else: uncond =  text_enc(neg_prompts, text.shape[1])
    emb = torch.cat([uncond, text])

    # 시드 설정
    if seed: torch.manual_seed(seed)

    # 무작위 노이즈 초기화
    latents = torch.randn((bs, unet.in_channels, dim//8, dim//8))

    # 스케줄러에서 스텝 수 설정
    scheduler.set_timesteps(steps)

    # 노이즈를 잠재 변수에 추가
    latents = latents.to("cuda").half() * scheduler.init_noise_sigma

    # 정의된 스텝 수를 반복
    for i,ts in enumerate(tqdm(scheduler.timesteps)):
        # 입력 잠재 변수를 분산과 일치하도록 스케일링해야 함
        inp = scheduler.scale_model_input(torch.cat([latents] * 2), ts)

        # U-Net을 사용하여 노이즈 잔여 값을 예측
        with torch.no_grad(): u,t = unet(inp, ts, encoder_hidden_states=emb).sample.chunk(2)

        # 가이던스 수행
        pred = u + g*(t-u)

        # 잠재 변수 조건
        latents = scheduler.step(pred, ts, latents).prev_sample

        # 중간 이미지 저장
        if save_int:
            if not os.path.exists(f'./steps'): os.mkdir(f'./steps')
            latents_to_pil(latents)[0].save(f'steps/{i:04}.jpeg')

    # 잠재 표현을 반환하여 3x512x512 크기의 이미지 생성
    return latents_to_pil(latents)

In [ ]:
images = prompt_2_img(prompts = ["beach,landscape,photograph,simple"]
                     ,neg_prompts=["body, hair, face, eyes, nose, ears, lip, legs, arms, hands, feet, girl, boy, people"]
                     ,steps=50, save_int=False)[0]
new_width, new_height = (1280, 720)  # 유튜브 썸네일 이미지 규격
images = images.resize((new_width, new_height), Image.ANTIALIAS)
plt.imshow(images)
plt.axis('off')
plt.show()

### 3. 이미지 저장

In [ ]:
# 저장할 폴더 경로 설정
output_folder = "result"

# 폴더가 존재하지 않으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 생성된 이미지 파일의 이름
output_file_name = f"{new_title}.jpg"

# 이미지 파일의 전체 경로 설정
output_file_path = os.path.join(output_folder, output_file_name)

In [ ]:
# 이미지 저장
images.save(output_file_path)